In [61]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [15]:
with open('../data/best_lr_retrained_tfidf.pckl', 'rb') as f:
    lr = pickle.load(f)

with open('../data/best_svm_retrained_tfidf.pckl', 'rb') as f:
    svm = pickle.load(f)

with open('../data/best_mnb_retrained_tfidf.pckl', 'rb') as f:
    mnb = pickle.load(f)

In [40]:
with open(f'../data/test_tfidf_subsampled_data_3.pckl', 'rb') as f:
    test_full = pickle.load(f)

with open(f'../data/test_tfidf_subsampled_data_sliced_2.pckl', 'rb') as f:
    test_sliced = pickle.load(f)

In [41]:
test_full.shape, test_sliced.shape

((72165, 2997374), (72165, 83235))

In [43]:
predicted_lr = lr.predict_proba(test_sliced)[:, 1]
predicted_svm = svm.predict_proba(test_full)[:, 1]
predicted_mnb = mnb.predict_proba(test_full)[:, 1]

with open('predicted_test_labels.pckl', 'wb') as f:
    pickle.dump((predicted_lr,
                 predicted_svm,
                 predicted_mnb),
                f)

In [44]:
predicted_lr[:10]

array([0.3965505 , 0.55176287, 0.63057129, 0.59796197, 0.5513886 ,
       0.51795874, 0.73287245, 0.42567349, 0.60286262, 0.58485538])

In [45]:
predicted_svm[:10]

array([0.39629447, 0.49678376, 0.64997718, 0.67418289, 0.58199489,
       0.40619709, 0.7068675 , 0.43030727, 0.55686505, 0.51671582])

In [46]:
predicted_mnb[:10]

array([0.295664  , 0.81981651, 0.34628548, 0.76782082, 0.85840238,
       0.00964206, 0.82809496, 0.12949945, 0.48101807, 0.50140059])

In [48]:
(predicted_lr > 0.5).sum(), (predicted_svm > 0.5).sum(), (predicted_mnb > 0.5).sum()

(26539, 25362, 26899)

In [53]:
ensemble_df = pd.DataFrame([predicted_lr, predicted_svm, predicted_mnb]).T
ensemble_df.columns = ['lr', 'svm', 'mnb']
ensemble_df['vote'] = ((ensemble_df[['lr', 'svm', 'mnb']] > 0.5).sum(axis=1) > 2).astype(int)
ensemble_df['vote_proba'] = (ensemble_df[['lr', 'svm', 'mnb']] > 0.5).sum(axis=1) / 3
ensemble_df['avg'] = ensemble_df[['lr', 'svm', 'mnb']].mean(axis=1)
ensemble_df.head()

,lr,svm,mnb,vote,vote_proba,avg
0,0.396550,0.396294,0.295664,0,0.000000,0.362836
1,0.551763,0.496784,0.819817,0,0.666667,0.622788
2,0.630571,0.649977,0.346285,0,0.666667,0.542278
3,0.597962,0.674183,0.767821,1,1.000000,0.679989
4,0.551389,0.581995,0.858402,1,1.000000,0.663929


In [56]:
ensemble_df['avg'].to_csv('../data/predictions.csv', index=False, header=False)